# I want to see which stars have surface rotation rates

In [1]:
import numpy as np
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import astropy.units as u
from astroquery.vizier import Vizier
from astropy.io import ascii
import warnings
warnings.filterwarnings('ignore')

sns.set_palette('colorblind')
sns.set_context('poster')

In [2]:
ati = pd.read_csv('../../data/atium.csv',index_col=0)


In [3]:
ati = pd.read_csv('../../data/atium.csv',index_col=0)
#Exclude those stars with a Rhat > 1.1 flag and no results otherwise
ati = ati[np.isfinite(ati.nus.values)]
ati = ati.loc[ati.flag != 'rhat_1' ]
print(f'We have {len(ati)} stars with Rhats below 1.1')

We have 82 stars with Rhats below 1.1


In [4]:
lit = pd.DataFrame()
lit['KIC'] = ati.KIC.copy()
lit['Flag'] = np.nan

### Read in the McQuillan+ sample

In [5]:
cat = Vizier(catalog='J/ApJS/211/24/table1', 
             columns=['KIC', 'Prot', 'e_Prot'])
cat.ROW_LIMIT = -1
mcq = cat.get_catalogs(cat.catalog)[0].to_pandas()
mcq = mcq.rename(columns={'Prot':'McQP','e_Prot':'e_McQP'})

In [6]:
mcq.head(2)

,KIC,McQP,e_McQP
0,892376,1.532,0.007
1,1026146,14.891,0.120


#### Merge results in

In [7]:
lit = pd.merge(lit, mcq, how='left', on='KIC')

In [8]:
lit.loc[np.isfinite(lit.McQP), 'Flag'] = 1

In [9]:
print(f'We have {len(lit[np.isfinite(lit["McQP"])])} / {len(lit)} periods from McQuillan+15')

We have 9 / 82 periods from McQuillan+15


### Read in the Santos+19 sample

In [10]:
santos19 = ascii.read('../../data/santos19.txt').to_pandas()
santos19 = santos19[['KIC', 'Prot', 'E_Prot']]
santos19 = santos19.rename(columns={'Prot':'SP','E_Prot':'e_SP'})

In [11]:
santos19.head(2)

,KIC,SP,e_SP
0,892834,13.61,1.02
1,892882,21.96,1.70


#### Merge results in

In [12]:
lit = pd.merge(lit, santos19, how='left', on='KIC')
lit.loc[np.isfinite(lit.SP), 'Flag'] = 1

In [13]:
print(f'We have {len(lit[np.isfinite(lit["SP"])])} / {len(lit)} periods from Santos+19')

We have 0 / 82 periods from Santos+19


### Read in the Nielsen+13 sample

In [14]:
cat = Vizier(catalog='J/A+A/557/L10/table1', 
             columns=['KIC', 'Prot', 'MAD'])
cat.ROW_LIMIT = -1
n13 = cat.get_catalogs(cat.catalog)[0].to_pandas()
n13 = n13.rename(columns={'Prot':'NP','MAD':'e_NP'})

In [15]:
n13.head(2)

,KIC,NP,e_NP
0,891916,5.539,0.191
1,892834,13.663,0.098


#### Merge results in

In [16]:
lit = pd.merge(lit, n13, how='left', on='KIC')
lit.loc[np.isfinite(lit.NP), 'Flag'] = 1

In [17]:
print(f'We have {len(lit[np.isfinite(lit["NP"])])} / {len(lit)} periods')

We have 6 / 82 periods


### Read in the van Saders+16 sample

In [18]:
jvs = pd.read_csv('../../data/jvs_rot.csv', index_col=None)
jvs.dropna(inplace=True)
jvs.KIC = jvs.KIC.astype(int)
jvs.head(2)

,KIC,jvsP
1,12069424,23.8
2,12069449,23.2


#### Merge results in


In [19]:
lit = pd.merge(lit, jvs, how='left', on='KIC')
lit.loc[np.isfinite(lit.jvsP), 'Flag'] = 1
print(f'We have {len(lit[np.isfinite(lit["jvsP"])])} / {len(lit)} periods from van Saders+16')

We have 18 / 82 periods from van Saders+16


In [21]:
lit.to_csv('../../data/lit_spot_rots.csv')